In [1]:
import sys
import os
import pandas as pd
from pathlib import Path

project_root = Path.cwd().parent  # one level up from "notebook"
sys.path.append(str(project_root))

parent_dir = os.path.abspath(os.path.join(os.getcwd(), ".."))

if parent_dir not in sys.path:
    sys.path.append(parent_dir)

train_df = pd.read_json("../data/raw/training_merged.json", orient="records", lines=True)
dev_df = pd.read_json("../data/raw/development_merged.json", orient="records", lines=True)
training_sample = 500
dev_sample = 200
train_df = train_df.sample(n=training_sample, random_state=42)
dev_df = dev_df.sample(n=dev_sample, random_state=42)


In [2]:
from project_name.preprocessing.BERTweet_preprocessing import BERTweetPreprocessor
preprocessor = BERTweetPreprocessor()
preprocessor.extract_features_labels(train_df, 'tweet', 'emotion')
X_train, y_train = preprocessor.preprocess_df(train_df)
X_dev, y_dev = preprocessor.preprocess_df(dev_df)

In [11]:
from transformers import AutoTokenizer
from project_name.data.tokenizer import TweetDataset
import torch
from torch.utils.data import DataLoader

tokenizer = AutoTokenizer.from_pretrained("vinai/bertweet-base", use_fast=False)
train_set = TweetDataset(X_train, y_train)
dev_set = TweetDataset(X_dev, y_dev)

def collate_batch(batch):
    tweets, labels = zip(*batch)
    encoding = tokenizer(
        list(tweets),
        padding=True,
        truncation=True,
        max_length=128,
        return_token_type_ids=False,
        return_tensors="pt"
    )
    labels = torch.tensor(labels, dtype=torch.long)
    encoding["labels"] = labels
    return encoding
num_labels = len(set(y_train))
train_loader = DataLoader(train_set, batch_size=32, shuffle=True, collate_fn=collate_batch)
dev_loader = DataLoader(dev_set, batch_size=32, collate_fn=collate_batch)


In [12]:
from transformers import AutoModelForSequenceClassification
import torch

model = AutoModelForSequenceClassification.from_pretrained("vinai/bertweet-base", num_labels=num_labels)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)


Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at vinai/bertweet-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


RobertaForSequenceClassification(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(64001, 768, padding_idx=1)
      (position_embeddings): Embedding(130, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-11): 12 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
         

In [ ]:
from torch.optim import AdamW
from tqdm.auto import tqdm

optimizer = AdamW(model.parameters(), lr=2e-5)
EPOCHS = 1
for epoch in range(EPOCHS):
    model.train()
    total_loss = 0

    for batch in tqdm(train_loader, desc=f"Epoch {epoch+1}/{EPOCHS}"):
        batch = {k: v.to(device) for k, v in batch.items()}

        outputs = model(**batch)
        loss = outputs.loss

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    print(f"Epoch {epoch+1}: train loss = {total_loss/len(train_loader):.4f}")

    # ---------- validation ----------
    model.eval()
    correct, total = 0, 0
    with torch.no_grad():
        for batch in dev_loader:
            batch = {k: v.to(device) for k, v in batch.items()}        
            inputs = {k: v for k, v in batch.items() if k != 'labels'}
            logits = model(**batch).logits
            preds = logits.argmax(dim=1)
            labels = batch["labels"]
            correct += (preds == labels).sum().item()
            total += labels.size(0)

    print(f"Epoch {epoch+1}: dev accuracy = {correct/total:.2%}")

Epoch 1/1:   0%|          | 0/16 [00:00<?, ?it/s]